<h1>Biblioteca</h1>

In [37]:
import pandas as pd
import numpy as np 

import datetime, pytz


<h1>Funções Coringas</h1>

In [2]:
##Exportação do dataframe em Python
##df_excel=df
##df_excel.to_excel (r'C:\Users\Gabriel\Desktop\Mestrado\export_dataframe.xlsx', index = False, header=True)

<h1>Pré-Processamento</h1>

In [54]:
## Dataframe Origenal
json = pd.read_json('https://jeap.rio.rj.gov.br/dadosAbertosAPI/v2/transporte/veiculos/onibus2')

#Removendo campos em branco
nan_value = float("NaN")
json.replace("", nan_value, inplace=True)
json.dropna(subset = ["linha"], inplace=True)
json['codigoempresa']=json['ordem'].str.slice(1,4)

#Novo Dataframe

ordem=json['ordem']
linha=json['linha']

empresa=[        "Transportes Paranapuan S/A" if s>="100" and s<"105" else
                  "Viação Penha Rio Ltda" if s>="105" and s<"115" else
                  "Viação Vila Real S/A" if s>="115" and s<"120" else
                  "Expresso Recreio Ltda" if s>="120" and s<"125" else
                  "Viação Normandy To Triângulo Ltda" if s>="125" and s<"130" else
                  "Transportes Barra Ltda" if s>="130" and s<"170" else 
                  "Auto Viação Palmares Ltda" if s>="170" and s<"255" else
                  "Roviária Âncora Matias" if s>="225" and s<"270" else
                  "Caprichosa Auto Ônibus Ltda" if s>="270" and s<"275" else
                  "Transportes Vila Isabel S/A " if s>="275" and s<"285" else
                  "Viação Ideal S/A" if s>="285" and s<"290" else
                  "Empresa Braso Lisboa" if s>="290" and s<"300" else 
                  "Transportes Futuro Ltda" if s>="300" and s<"310" else 
                  "Viação VG Eireli ME" if s>="310" and s<"325" else 
                  "Viação Pavuenense S/A" if s>="325" and s<"410" else 
                  "Real Auto Ônibus Ltda" if s>="410" and s<"425" else 
                  "Viação Acari S/A" if s>="425" and s<"445" else 
                  "Auto Viação Três Amigos Ltda" if s>="445" and s<"475" else
                  "Viação Redentor Ltda" if s>="475" and s<"480" else
                  "Auto Viação Alpha S/A " if s>="480" and s<"500" else
                  "Tijuquinha - Auto Viação Tijuca S/A" if s>="500" and s<"515" else
                  "Viação Novacap S/A" if s>="515" and s<"535" else
                  "Transportes Campo Grande Ltda" if s>="535" and s<"580" else
                  "Viação Nossa Senhora de Lourdes Ltda" if s>="580" and s<"630" else
                  "Gire Transportes Ltda" if s>="630" and s<"710" else
                  "Viação Verdun S/A" if s>="710" and s<"715" else
                  "Viação Nossa Senhora das Graças Ltda" if s>="715" and s<"720" else
                  "Transurb S/A" if s>="720" and s<"860" else
                  "Auto Viação Jabour Ltda" if s>="860" and s<"870" else
                  "Expresso Pégasso Ltda" if s>="870" and s<"999" 
                   else "BRT Rio S.A"  for s in json['codigoempresa']
] 

codigoempresa=json['ordem'].str.slice(1,4)
latitude=json['latitude']
longitude=json['longitude']
velocidade=json['velocidade']

#Datas e Horas
datastring=json['dataHora'].str.slice(0,10).astype('datetime64[ns]')
data=datastring.dt.strftime('%d/%m/%Y')
hora=json['dataHora'].str.slice(11,19)
dataHora=json['dataHora'].astype('datetime64[ns]')

df2= pd.DataFrame(list(zip(ordem,linha,empresa,latitude,longitude,velocidade,data,hora,dataHora)),columns=['ordem','linha','empresa','latitude','longitude','velocidade','data','hora','dataHora'])

horamaluca=df2['dataHora']
df2['jordi']=pd.to_datetime(horamaluca, unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Argentina/Catamarca')

df2


,ordem,linha,empresa,latitude,longitude,velocidade,data,hora,dataHora,jordi
0,D13190,383,Transportes Barra Ltda,-22.87630,-43.42000,0,03/06/2022,01:19:24,2022-06-03 01:19:24,2022-06-02 22:19:24-03:00
1,B32820,665,Viação Pavuenense S/A,-22.81605,-43.39466,0,03/06/2022,02:33:14,2022-06-03 02:33:14,2022-06-02 23:33:14-03:00
2,C47664,863,Viação Redentor Ltda,-22.97870,-43.32630,42,03/06/2022,04:06:16,2022-06-03 04:06:16,2022-06-03 01:06:16-03:00
3,C13039,878,Transportes Barra Ltda,-22.87991,-43.35764,0,03/06/2022,07:08:42,2022-06-03 07:08:42,2022-06-03 04:08:42-03:00
4,C30262,565,Transportes Futuro Ltda,-22.94984,-43.34782,0,03/06/2022,07:14:22,2022-06-03 07:14:22,2022-06-03 04:14:22-03:00
...,...,...,...,...,...,...,...,...,...,...
3507,B71108,457,Viação Verdun S/A,-22.88861,-43.27477,0,03/06/2022,14:08:12,2022-06-03 14:08:12,2022-06-03 11:08:12-03:00
3508,B71020,455,Viação Verdun S/A,-22.98676,-43.19081,0,03/06/2022,14:08:13,2022-06-03 14:08:13,2022-06-03 11:08:13-03:00
3509,B71136,455,Viação Verdun S/A,-22.90493,-43.18313,2,03/06/2022,14:08:13,2022-06-03 14:08:13,2022-06-03 11:08:13-03:00
3510,B71025,238,Viação Verdun S/A,-22.92439,-43.25764,36,03/06/2022,14:08:14,2022-06-03 14:08:14,2022-06-03 11:08:14-03:00


In [4]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3492 entries, 0 to 3491
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ordem       3492 non-null   object        
 1   linha       3492 non-null   object        
 2   empresa     3492 non-null   object        
 3   latitude    3492 non-null   float64       
 4   longitude   3492 non-null   float64       
 5   velocidade  3492 non-null   int64         
 6   data        3492 non-null   object        
 7   hora        3492 non-null   object        
 8   dataHora    3492 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 245.7+ KB


In [52]:
df2.query("(linha=='770') and (velocidade!=0)")

,ordem,linha,empresa,latitude,longitude,velocidade,data,hora,dataHora,jordi
198,D17115,770,Auto Viação Palmares Ltda,-22.88809,-43.55970,25,03/06/2022,13:41:25,2022-06-03 13:41:25,2022-06-03 09:41:25-04:00
199,D17115,770,Auto Viação Palmares Ltda,-22.88809,-43.55970,25,03/06/2022,13:41:25,2022-06-03 13:41:25,2022-06-03 09:41:25-04:00
752,D17163,770,Auto Viação Palmares Ltda,-22.85529,-43.48230,68,03/06/2022,14:02:37,2022-06-03 14:02:37,2022-06-03 10:02:37-04:00
788,D17171,770,Auto Viação Palmares Ltda,-22.88924,-43.55989,4,03/06/2022,14:02:47,2022-06-03 14:02:47,2022-06-03 10:02:47-04:00
2125,D17165,770,Auto Viação Palmares Ltda,-22.86987,-43.55462,24,03/06/2022,14:03:30,2022-06-03 14:03:30,2022-06-03 10:03:30-04:00
2128,D17165,770,Auto Viação Palmares Ltda,-22.86987,-43.55462,24,03/06/2022,14:03:30,2022-06-03 14:03:30,2022-06-03 10:03:30-04:00
2381,D17168,770,Auto Viação Palmares Ltda,-22.86117,-43.44991,39,03/06/2022,14:03:33,2022-06-03 14:03:33,2022-06-03 10:03:33-04:00
2383,D17168,770,Auto Viação Palmares Ltda,-22.86117,-43.44991,39,03/06/2022,14:03:33,2022-06-03 14:03:33,2022-06-03 10:03:33-04:00
2882,D17159,770,Auto Viação Palmares Ltda,-22.90125,-43.55613,23,03/06/2022,14:03:42,2022-06-03 14:03:42,2022-06-03 10:03:42-04:00
3019,D17153,770,Auto Viação Palmares Ltda,-22.86269,-43.54209,1,03/06/2022,14:03:45,2022-06-03 14:03:45,2022-06-03 10:03:45-04:00


In [50]:
# horamaluca=df2['dataHora']
# pd.to_datetime(horamaluca, unit='ms').dt.tz_localize('UTC').dt.tz_convert('US/Eastern')

0      2022-06-02 21:19:24-04:00
1      2022-06-02 22:33:14-04:00
2      2022-06-03 00:06:16-04:00
3      2022-06-03 02:44:19-04:00
4      2022-06-03 03:08:42-04:00
                  ...           
3504   2022-06-03 09:50:15-04:00
3505   2022-06-03 09:50:15-04:00
3506   2022-06-03 09:50:15-04:00
3507   2022-06-03 09:50:15-04:00
3508   2022-06-03 09:50:17-04:00
Name: dataHora, Length: 3509, dtype: datetime64[ns, US/Eastern]